#### General Code Setup
As a first step we:
- Import all the necessary libraries for our computation
- Setup some folders for the loading and saving of all necessary files and to run correctly EMUstack

#### Libraries imported
- [os](https://docs.python.org/3/library/os.html): used to setup some folder to run correctly the code
- [sys](https://docs.python.org/3/library/sys.html): used to make EMUstack accessible
- [numpy](https://numpy.org/): essentially matlab for python. We use it to manipulate matrix data
- [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html): to run the code in parallel
- [xarray](https://docs.xarray.dev/en/stable/): library to save labelled data along with the relevant metadata (important for reproducibility)
- [datetime](https://docs.python.org/3/library/datetime.html): we use it to create a timestamp for the saved file

In [32]:
# libraries
import os
import numpy as np
import sys
import concurrent.futures
import xarray as xr
from datetime import datetime

# folder setup
home_dir = os.environ["HOME"]  # home folder
script_dir = os.getcwd()  # running file folder

#### Import EMUstack
In order to correctly load all the components of EMUstack we:
- make the EMUstack folder accessible
- load the EMUstack paths module to define the working folders
- load all the other EMUstack modules

In [33]:
# setup EMUstack paths
sys.path.append(home_dir + "/programs/EMUstack/backend/")  # makes EMUstack accessible
import paths
paths.backend_path = home_dir + "/programs/EMUstack/backend/"  # location of EMUstack "engine"
paths.data_path = home_dir + "/programs/EMUstack/backend/data/"  # location of material files
paths.msh_path = script_dir + "/msh/"  # folder where we save the mesh files
paths.template_path = home_dir + "/programs/EMUstack/backend/fortran/msh/"  # folder containing the mesh templates

# import emustack
import objects  # EMUstack incident field and layer structures
import materials  # EMUstack available material library
import plotting  # EMUstack plotting routins
from stack import Stack  # EMUstack multilayer structures

#### Defining the inputs
We are going to calculate the transmission and reflectance spectrum of a Gold Nanodisk array. To do so we need to define the following groups of input parameters:
- **Geometrical parameters** of our multilayers, including thicknesses, disk size, etc...
- **Material parameters** such as the composition of each layer, substrate and superstrate included
- **Illumination parameters**, i.e. wavelengths, angles and polarizations of the incident field
- **EMUstack parameters**, i.e. parameters that are strictly connected with the inner workings of EMUstack

We will try to store all these input parameters inside [dictionaries](https://docs.python.org/3/tutorial/datastructures.html#dictionaries), so that they can be handily exported later in order to improve the riproducibility of our calculations

In [34]:
# light dictionary
light = {}
light['theta'] = 0.0
light['phi'] = 0.0
light['pol'] = 'TM'
light['max_order_PWs'] = 2  # number of plane waves

# materials dictionary
mats = {}
mats['Au'] = materials.Au
mats['Air'] = materials.Air

# Geometrical parameters
struct = {}
# nanodisk array parameters
struct['periodicity'] = "2D_array"  # we are modeling a 1d array (EMU)
struct['inc_shape'] = 'circle'  # nanodisks -> so we choose the circle shape (EMU)
struct['nd_radius'] = 100.0  # nanodisk radius
struct['nd_height'] = 100.0  # nanodisk height
struct['nd_period_x'] = 600.0  # nanodisk array x period
struct['nd_period_y'] = 600.0  # nanodisk array y period
struct['nd_inc_mat'] = mats['Au']  # nanodisk material
struct['nd_back_mat'] = mats['Air']  # background material in the nanodisk array
struct['nd_loss'] = True  # we must include losses in the computation (EMU)
# superstrate and substrate parameters
struct['sub_mat'] = mats['Air']  # substrate material
struct['sup_mat'] = mats['Air']  # superstrate material
struct['sub_loss'] = False  # no losses in the substrate (EMU)
struct['sup_loss'] = False  # no losses in the superstrate (EMU)

# Emustack parameters
emu = {}
emu['plotting_fields'] = True  # Now WE DO WANT to plot the fields in this case 
emu['make_mesh_now'] = True  # we make the mesh, otherwise we must provide one with a filename
emu['force_mesh'] = True  # create mesh even if one already exist
emu['lc_bkg'] = 0.15  # background mesh finesse
emu['lc2'] = 2.0  # first mesh refinement
emu['lc3'] = 2.0  # second mesh refinement

#### Load spectral data
We load the spectra data. We then take the wavelength of maximum reflectance to compute the local fields

In [35]:
emu_spectra = xr.load_dataarray('data/EMU_AuNdArraySpectra_circle_px600.0_py400.0_r100.0_h100.0_180222_121902.nc')  # load the spectral data
wl_index = emu_spectra.sel(spectrum='R').argmax().item()  # find the index where the reflectance is maximum
light['wl'] = emu_spectra.coords['wavelength'][wl_index].item()  # get the wavelength of maximum reflactance
light_wl = objects.Light(light['wl'], max_order_PWs=light['max_order_PWs'], theta=light['theta'], phi=light['phi'])  # create and EMUstack incident light object at the correct wavelength

#### Initialize the single layers
In EMUstack each layer is initialized (and solved) separately and the solution is not influenced by its position inside the stack. There are two kind of layers:
- ThinFilm: layer that are not nanostructured and are simple to initialize
- NanoStruct: patterned layers which need more parameters to be initialized

In [36]:
# initialize substrate and superstrate
substrate = objects.ThinFilm(
    period=struct["nd_period_x"], period_y=struct["nd_period_y"], height_nm="semi_inf", material=struct["sub_mat"], loss=struct["sub_loss"]
)
superstrate = objects.ThinFilm(
    period=struct["nd_period_x"], period_y=struct["nd_period_y"], height_nm="semi_inf", material=struct["sup_mat"], loss=struct["sup_loss"]
)

# initialize nanodisk array layer
NDs = objects.NanoStruct(
    struct['periodicity'],  # 1d or 2d periodicity
    struct["nd_period_x"],  # structure period, in only one period, then square lattice
    2.0 * struct["nd_radius"],  # diameter of the inclusion
    period_y=struct["nd_period_y"],  # period along y axis
    height_nm=struct["nd_height"],  # nanostructured layer thickness
    inclusion_a=struct["nd_inc_mat"],  # composition of the inclusion (Au)
    background=struct["nd_back_mat"],  # composition of the background (Air)
    loss=struct['nd_loss'],  # Loss flag, to decide if we include losses in the computation
    inc_shape=struct['inc_shape'],  # shape of the inclusion
    plotting_fields=emu['plotting_fields'],  # we preserve the eigenmodes to plot the fields?
    make_mesh_now=emu['make_mesh_now'],  # are we making the mesh now or not?
    force_mesh=emu['force_mesh'],  # are we making the mesh even if one already exists?
    lc_bkg=emu['lc_bkg'],  # mesh general finesse
    lc2=emu['lc2'],  # first level of mesh refinement
    lc3=emu['lc3'],  # second level of mesh refinement
)



#### LF computation
Now we build functions that:
- takes and incident field, i.e. and object.Light as input
- solves for the mode in each layer, where the FEM solver works in the nanostructured layer
- builds a stack with the solver layer, in the order bottom -> top
- returns the stack
- take the returned stack as input and compute the local field input

In [37]:
# EMUstack Function
def simulate_stack(light):

    # evaluate each layer individually
    sim_NDs = NDs.calc_modes(light)  # solving the eigenproblem for the nanodisk layer
    sim_superstrate = superstrate.calc_modes(light)  # solving the plane wave problem for the superstrate
    sim_substrate = substrate.calc_modes(light)  # solving the plane wave problem for the substrate

    # build the stack with the solved layers and return it
    stackSub = Stack((sim_substrate, sim_NDs, sim_superstrate))

    return stackSub


def lf(stack, pol="TE", z=0.0, n_points=500, lay_interest=1):

    # explicitly compute the solution for the given polarization
    stack.calc_scat(pol = pol)

    # compute the fields interpolator in the layer of interest, at the height of interest
    ReEx, ImEx, ReEy, ImEy, ReEz, ImEz, AbsE = plotting.fields_interpolator_in_plane(stack, lay_interest=1, z_value=z)

    # create the x and y coordinates for the field mapping
    n_points = 500
    v_x = np.zeros(n_points ** 2)
    v_y = np.zeros(n_points ** 2)
    i = 0
    x_min = 0.0
    x_max = 1.0
    y_min = -(struct["nd_period_y"]/struct["nd_period_x"])
    y_max = 0.0
    for x in np.linspace(x_min, x_max, n_points):
        for y in np.linspace(y_min, y_max, n_points):
            v_x[i] = x
            v_y[i] = y
            i += 1
    v_x = np.array(v_x)
    v_y = np.array(v_y)

    # compute the interpolated fields over the grid
    m_AbsE = AbsE(v_x, v_y).reshape(n_points, n_points).T

    return m_AbsE



In [38]:
# simulate the single stack at the correct wavelength
stack_wl = simulate_stack(light_wl)

In [39]:
# compute the solution at the single good wavelength
m_AbsE = lf(stack_wl,pol = light['pol'],z = 0.5 *struct['nd_height'])

#### Saving data and metadata
We recast all the data in a format suitable for saving. We also save all the computation parameters as metadata, in order to make the computation of the spectrum reproducible

In [40]:
# build output filename
now = datetime.now()  # what time is it?
timestamp = now.strftime("%d%m%y_%H%M%S")  # build timestamp string as day,month,year _ hour,minute,second
out_filename = (
    "EMU_AuNdArrayLF_"
    + struct["inc_shape"]
    + "_px"
    + str(struct["nd_period_x"])
    + "_py"
    + str(struct["nd_period_y"])
    + "_r"
    + str(struct["nd_radius"])
    + "_h"
    + str(struct["nd_height"])
    + "_"
    + timestamp
)

# create vector to map x and y coords
v_coords_x = np.linspace(0.0,struct['nd_period_x'],500)
v_coords_y = np.linspace(0.0,struct['nd_period_y'],500)
# save the data as in the case of the spectra
emu_lf = xr.DataArray(m_AbsE, coords=[v_coords_y,v_coords_x], dims=["y", "x"])
emu_lf.attrs['light'] = str(light)
emu_lf.attrs['struct'] = str(struct)
emu_lf.attrs['emu'] = str(emu)
emu_lf.attrs['mats'] = str(mats)
emu_lf.to_netcdf('data/' + out_filename +  '.nc')